In [2]:
from turkish_morphology import decompose, analyze

word = "kitaplarımızdaki"
word = "yapamadıklarımızdan"
sentence = "Yazarın kitaplarını yayın hakkını elinde bulunduran Aziz Nesin Vakfı'ndan yapılan açıklamaya göre,ilki 1946 yılında basılan Aziz Nesin'in kitapları, o dönemden bu yana 10 milyon 780 bin adet basıldı ve dağıtıldı"
word = "dersindeyim"

analyses = analyze.surface_form(word, use_proper_feature=False)
decomposition = decompose.human_readable_analysis(analyses[0])

analyses
# decomposition

['(de[VB]+[Polarity=Pos])([VN]-r[Derivation=AorNom]+[PersonNumber=A3sg]+[Possessive=Pnon]+[Case=Bare])([NOMP]-sH[Derivation=Sim]+[PersonNumber=A3sg]+Hn[Possessive=P2sg]+NDA[Case=Loc]+[Copula=PresCop]+YHm[PersonNumber=V1sg])',
 '(ders[NOMP]+[PersonNumber=A3sg]+Hn[Possessive=P2sg]+NDA[Case=Loc]+[Copula=PresCop]+YHm[PersonNumber=V1sg])',
 '(ders[NOMP]+[PersonNumber=A3sg]+SH[Possessive=P3sg]+NDA[Case=Loc]+[Copula=PresCop]+YHm[PersonNumber=V1sg])']

In [3]:
from morphology import decompose_tr

decomposition = decompose_tr(word)
decomposition

[Decomposition(root=ders, pos=NOMP, meta_morphemes=['SH', 'NDA', 'YHm'], morphemes=['i', 'nde', 'yim']),
 Decomposition(root=ders, pos=NOMP, meta_morphemes=['Hn', 'NDA', 'YHm'], morphemes=['in', 'de', 'yim']),
 Decomposition(root=de, pos=NOMP, meta_morphemes=['r', 'sH', 'Hn', 'NDA', 'YHm'], morphemes=['r', 'si', 'n', 'de', 'yim']),
 Decomposition(root=de, pos=NOMP, meta_morphemes=['r', 'sH', 'Hn', 'NDA', 'YHm'], morphemes=['r', 'si', 'n', 'de', 'yim'])]

In [ ]:
import pandas as pd

data = pd.read_csv("data/bilkent-turkish-writings/bilkent-turkish-writings-dataset.csv")
data.head()

In [ ]:
import re
from utils import write_json

essays = []

for i, row in data.iterrows():
    if isinstance(row["text"], str):
        raw_texts = re.split("[.?!]\s",row["text"].replace("\r\n", ""))
        texts = []

        for text in raw_texts:
            texts.extend(re.split("\s\s\s\s", text))

        texts = [re.sub("^[A-ZÜÖĞIŞÇİ0-9\-\s.]+$", "", text) for text in texts]
        texts = [re.sub("\s+", " ", text).strip() for text in texts if all(["http" not in text.lower(), "kaynakça" not in text.lower()]) and len(text.strip().split()) > 5]
        
        if len(texts) > 1:
            essays.append({
                "id": f"bilkent-turkish-writings-{i}",
                "sentences": texts
            })

write_json(essays, "data/bilkent-turkish-writings/bilkent-turkish-writings-dataset.json", ensure_ascii=False)


In [ ]:
from morphology import decompose_tr

# decompose_tr("heykelciklerinin")
[decomposition.to_json() for decomposition in decompose_tr("yansıttığımızın")]

In [ ]:
from utils import read_json

data = read_json("../data/bilkent-turkish-writings/bilkent-turkish-writings-dataset_prep.json")

In [ ]:
len(data), sum([len(val) for val in data.values()])

In [ ]:
sum([1 for root, word_map in data.items() for word, decompositions in word_map.items() if len(decompositions) == 1 and decompositions[0]["morphemes"]])

In [ ]:
from utils import write_json
import pathlib

pathlib.Path("../data/bilkent-turkish-writings/batches").mkdir(parents=True, exist_ok=True)
i = 0
while True:
    data_chunk = list(data.items())[i*500:(i+1)*500]
    if data_chunk:
        write_json(dict(data_chunk), f"../data/bilkent-turkish-writings/batches/bilkent-turkish-writings-dataset_prep_{i}.json", ensure_ascii=False)
        i += 1
    else:
        break

In [1]:
from utils import write_json
from prepare_data_for_morph import post_morph_process

new_data = post_morph_process("../data/bilkent-turkish-writings/batches/bilkent-turkish-writings-dataset_prep_0.json")

write_json(new_data, "../data/bilkent-turkish-writings/batches/bilkent-turkish-writings-dataset_prep_0_new.json", ensure_ascii=False)